<a href="https://colab.research.google.com/github/dineshkumarsarangapani/Machine_learning_IITM/blob/main/MLP/week2_graded.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

worksheet = gc.open('mlp-week2-data').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
print(rows)

# Convert to a DataFrame and render.
import pandas as pd
data = pd.DataFrame.from_records(rows, columns=['V1','V2','V3','V4','V5','Target'], coerce_float=True)

[['2', '50', '12500', '98', 'NEGATIVE', 'YES'], ['0', '13', '3250', '28', 'NEGATIVE', 'YES'], ['', '', '4000', '35', 'NEGATIVE', 'YES'], ['', '20', '5000', '45', 'NEGATIVE', 'YES'], ['1', '24', '6000', '77', 'NEGATIVE', 'NO'], ['4', '', '1000', '4', 'NEGATIVE', 'NO'], ['2', '', '1750', '14', 'NEGATIVE', 'YES'], ['', '12', '3000', '35', 'NEGATIVE', 'NO'], ['2', '9', '2250', '22', 'NEGATIVE', 'YES'], ['5', '46', '11500', '98', 'NEGATIVE', 'YES'], ['4', '', '5750', '58', 'NEGATIVE', 'NO'], ['', '3', '750', '4', 'NEGATIVE', 'NO'], ['2', '10', '2500', '28', 'NEGATIVE', 'YES'], ['1', '13', '3250', '47', 'NEGATIVE', 'NO'], ['2', '6', '1500', '15', 'NEGATIVE', 'YES'], ['2', '5', '1250', '11', 'NEGATIVE', 'YES'], ['', '14', '3500', '48', 'NEGATIVE', 'YES'], ['2', '', '3750', '49', 'NEGATIVE', 'YES'], ['2', '6', '1500', '15', 'NEGATIVE', 'YES'], ['2', '3', '750', '4', 'NEGATIVE', 'YES'], ['2', '3', '750', '4', 'NEGATIVE', 'YES'], ['4', '11', '2750', '28', 'NEGATIVE', 'NO'], ['2', '6', '1500', '1

In [ ]:
import numpy as np
data = data.replace(r'^\s*$', np.nan, regex=True)

In [ ]:
data.head()

,V1,V2,V3,V4,V5,Target
0,2,50,12500,98,NEGATIVE,YES
1,0,13,3250,28,NEGATIVE,YES
2,NaN,NaN,4000,35,NEGATIVE,YES
3,NaN,20,5000,45,NEGATIVE,YES
4,1,24,6000,77,NEGATIVE,NO


In [ ]:
data.shape

(748, 6)

In [ ]:
data.isnull().sum()

V1        5
V2        5
V3        0
V4        0
V5        0
Target    0
dtype: int64

In [ ]:
data.dtypes

V1        object
V2        object
V3        object
V4        object
V5        object
Target    object
dtype: object

In [ ]:
data['V1'] = pd.to_numeric(data['V1'])
data['V2'] = pd.to_numeric(data['V2'])
data['V3'] = pd.to_numeric(data['V3'])
data['V4'] = pd.to_numeric(data['V4'])
data.dtypes

V1        float64
V2        float64
V3          int64
V4          int64
V5         object
Target     object
dtype: object

In [ ]:
data['Target'].value_counts()

NO     570
YES    178
Name: Target, dtype: int64

In [ ]:
data['V5'].value_counts()

NEGATIVE    748
Name: V5, dtype: int64

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn import set_config
set_config(display='diagram')

In [ ]:
print(type(data))
data.columns

<class 'pandas.core.frame.DataFrame'>


Index(['V1', 'V2', 'V3', 'V4', 'V5', 'Target'], dtype='object')

In [75]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.feature_selection import VarianceThreshold


num_pipeline = Pipeline(
    [
     ("features_v1", ColumnTransformer(transformers=[("simple_imputer", SimpleImputer(strategy='median'), [0, 1])],  remainder='passthrough')),
     ("features_v2", ColumnTransformer([("standard_scale", StandardScaler(), [0,1,2,3])
     ]))
    ])

ord_pipeline = Pipeline([
    ("targets",ColumnTransformer([("ordinal_endocder", OrdinalEncoder(), [4])]))
])
# ct = ColumnTransformer(
#    [("simple_imputer", SimpleImputer(strategy='median'), [0, 1]),
#    ("standard_scale", StandardScaler(), [0, 1,2,3]),
#     ("ordinal_endocder", OrdinalEncoder(), [4])
#    ])



full_pipeline = FeatureUnion(transformer_list=[("num",num_pipeline), ('ord', ord_pipeline)])
full_pipeline


FeatureUnion(transformer_list=[('num',
                                Pipeline(steps=[('features_v1',
                                                 ColumnTransformer(remainder='passthrough',
                                                                   transformers=[('simple_imputer',
                                                                                  SimpleImputer(strategy='median'),
                                                                                  [0,
                                                                                   1])])),
                                                ('features_v2',
                                                 ColumnTransformer(transformers=[('standard_scale',
                                                                                  StandardScaler(),
                                                                                  [0,
                                                                                   1,
                                                                                   2,
                                                                                   3])]))])),
                               ('ord',
                                Pipeline(steps=[('targets',
                                                 ColumnTransformer(transformers=[('ordinal_endocder',
                                                                                  OrdinalEncoder(),
                                                                                  [4])]))]))])

In [76]:
print(data.shape)
X = full_pipeline.fit_transform(data)
print(X.shape)
# print(X)



(748, 6)
(748, 5)


In [90]:
y_df = pd.DataFrame(data['Target'])
ord_target = OrdinalEncoder()
y = ord_target.fit_transform(y_df)
print(y.shape)
y = y.ravel()
print(y.shape)

(748, 1)
(748,)


In [91]:
variance_threshold =  VarianceThreshold(threshold=0.1)
X_vt = variance_threshold.fit_transform(X)
print(X_vt.shape)

(748, 4)


In [92]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE

estimator = LogisticRegression()
selector = RFE(estimator)
selector.fit(X, y)
selector.get_feature_names_out(['V1', 'V2', 'V3', 'V4', 'V5'])

array(['V1', 'V3'], dtype=object)

In [95]:
from sklearn.feature_selection import SequentialFeatureSelector

sfs = SequentialFeatureSelector(estimator, direction='forward')
sfs.fit(X, y)
sfs.get_feature_names_out(['V1', 'V2', 'V3', 'V4', 'V5'])

array(['V2', 'V4'], dtype=object)

In [96]:
from sklearn.feature_selection import SequentialFeatureSelector

sfs = SequentialFeatureSelector(estimator, direction='backward')
sfs.fit(X, y)
sfs.get_feature_names_out(['V1', 'V2', 'V3', 'V4', 'V5'])

array(['V3', 'V4'], dtype=object)